# Auto load books


In [4]:
#!cd /home/youssef/Workspace/projects/schoollibrary/schoollibrary



In [2]:
!pwd

/home/youssef/Workspace/projects/schoollibrary/schoollibrary


In [1]:
!source bpslibrary/bin/activate

from urllib import request as urllib_request
from time import sleep
import re
import sqlite3
from flask import Blueprint, flash, redirect, render_template, request
#from flask_login import current_user
#from flask_paginate import Pagination, get_page_parameter
from sqlalchemy import exc, or_
from bpslibrary import app
from bpslibrary.database import db_session
from bpslibrary.forms import NewLoanForm, LoanReturnForm
from bpslibrary.models import Author, Book, Category, Pupil
from bpslibrary.utils.barcode import scan_for_isbn
from bpslibrary.utils.apihandler import APIClient
from bpslibrary.utils.permission import admin_access_required
from bpslibrary.utils.enums import BookLocation


2018-01-17 20:36:37,518 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-01-17 20:36:37,519 INFO sqlalchemy.engine.base.Engine ()
2018-01-17 20:36:37,520 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-01-17 20:36:37,521 INFO sqlalchemy.engine.base.Engine ()
2018-01-17 20:36:37,522 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("book_author")
2018-01-17 20:36:37,523 INFO sqlalchemy.engine.base.Engine ()
2018-01-17 20:36:37,525 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("book_category")
2018-01-17 20:36:37,525 INFO sqlalchemy.engine.base.Engine ()
2018-01-17 20:36:37,526 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("classrooms")
2018-01-17 20:36:37,527 INFO sqlalchemy.engine.base.Engine ()
2018-01-17 20:36:37,528 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("pupils")
2018-01-17 20:36:37,528 INFO sqlalchemy.engine.base.Engine ()
2018-01-17 20:36:37,529 IN

In [3]:

def auto_load_books():
    """Automated book loading.

    Combining the lookup and add functions, this function aims at bulk
    loading of books in the background.
    """
    session = db_session()
    lookup_limit = request.args.get('n')

    # lookup books
    succeeded = []
    failed = []

    try:
        for isbn in execute_sql('fetch_isbn', lookup_limit):
            api_client = APIClient(isbn, None)
            found_books = api_client.find_books(direct_search_only=True)

            # ensure we don't exceed google api usage limit
            sleep(20)

            # to ensure only the right book is added, only search resulting
            # yielding 1 result is accepted.
            if len(found_books) != 1:
                failed.append(
                    (isbn[0], "search results %d" % len(found_books)))
                continue

            # now we add the book.
            book = found_books[0]

            # defaults
            book.is_available = True
            book.current_location = BookLocation.LIBRARY.value

            if book.thumbnail_url:
                image_name = ''.join(
                    [c for c in book.title.replace(' ', '_')
                     if re.match(r'\w', c)]) + book.isbn13 + '.jpg'
                img = open(THUMBNAILS_ABSOLUTE_DIR + image_name, 'wb')
                img.write(urllib_request.urlopen(book.thumbnail_url).read())
                book.image_name = image_name

            for i in range(len(book.authors)):
                lookup_author = Author.query.filter(
                    Author.name == book.authors[i].name).first()
                if lookup_author:
                    book.authors[i] = None
                    book.authors[i] = lookup_author

            for i in range(len(book.categories)):
                lookup_category = Category.query.filter(
                    Category.name == book.categories[i].name).first()
                if lookup_category:
                    book.categories[i] = None
                    book.categories[i] = lookup_category

            session.add(book)
            session.commit()
            succeeded.append(isbn[0])

        execute_sql('update_success', succeeded=succeeded)
        execute_sql('update_failed', failed=failed)

    except RuntimeError as rt_error:
        print(str(rt_error))

In [4]:
def execute_sql(action, lookup_limit=10, failed=None, succeeded=None):
    """Execute sql to fetch isbns to lookup or update lookup status."""
    # update the lookup table
    connection = sqlite3.connect(
        app.config['DATABASE_URI'].replace('sqlite:///', ''))
    cursor = connection.cursor()
    sql = ""
    if action == 'fetch_isbn':
        sql = """
            SELECT DISTINCT isbn
            FROM isbn_lookup
            WHERE status IS NULL
            OR status NOT LIKE '%SUCCESS%'
            ORDER BY last_check
            LIMIT {0};
            """.format(int(lookup_limit))
        cursor.execute(sql)
        return cursor.fetchall()

    if action == 'update_success':
        assert (succeeded is not None), "Succeeded list is None."
        sql = """
            UPDATE isbn_lookup
            SET
                status = 'SUCCESS_DIRECT',
                last_check = CURRENT_TIMESTAMP
            WHERE isbn in ('{0}');
            """.format("','".join(str(i) for i in succeeded))
        cursor.execute(sql)
        connection.commit()

    if action == 'update_failed':
        assert (failed is not None), "Failed list is None."
        sql = """
            UPDATE isbn_lookup
                SET errors = CASE isbn """
        for isbn, error in failed:
            sql += " WHEN '%s' THEN errors + '|%s'" % (isbn, error)
        sql += """ ELSE errors END,
            last_check = CURRENT_TIMESTAMP,
            status = 'FAILED_DIRECT'
            WHERE isbn IN ('%s');""" % "','".join([str(i) for i, e in failed])
        cursor.execute(sql)
        connection.commit()

    connection.close()


In [5]:
auto_load_books()

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.